In [1]:
%load_ext autoreload

In [2]:
#!pip3 install --upgrade flair

In [104]:
print('hello')

hello


In [3]:
%autoreload 2

In [4]:
from utils.dataset_download import *
import pickle
import json
from pathlib import Path, PosixPath
import pandas as pd

In [5]:
# import API credentials
with open('oed_experiments/oed_credentials.json') as f:
    credentials = json.load(f)

In [6]:
# define lemma
lemma_id = "machine_nn01"

In [7]:
save_path = Path("./data")
save_path.mkdir(exist_ok=True)

In [ ]:
#query the API and get the json response
sense_json = query_oed(credentials,'word',lemma_id,flags='include_senses=true&include_quotations=true')

# convert the json in a dataframe
senses_df = convert_json_to_dataframe(sense_json)

In [ ]:
# save the dataframe
# as pickle
senses_df.to_pickle(save_path / f"senses_{lemma_id}.pickle")
# as csv
senses_df.to_csv(save_path / f"senses_{lemma_id}.tsv",sep='\t')

In [ ]:
# open pickle file to avoid calling the API again
with open(save_path / f"senses_{lemma_id}.pickle",'rb') as in_pickle:
    machine_senses_df = pickle.load(in_pickle)

In [ ]:
# get all senses that are siblings and descendants
# of the semantic class of senses listed in previously obtained query 
responses = traverse_thesaurus(credentials,machine_senses_df)

In [ ]:
# traverse tree or load responses 
# responses = traverse_thesaurus(credentials,machine_senses_df)
with open('./data/tree_traversal.pickle','rb') as in_pickle:
    responses = pickle.load(in_pickle)

In [ ]:
# get all quoations for the senses in the responses variable
quotations = get_quotations_from_thesaurus(credentials,responses)

In [ ]:
# merge and save all information stored in the seperate pickle files
df = merge_pickled(Path("./data/senses_machine_nn01.pickle"),
                   Path("./data/tree_traversal.pickle"),
                   Path("./data/tree_traversal_quotations.pickle"))

In [ ]:
df.to_pickle(f"./data/{lemma_id}_all.pickle")

In [ ]:
df.head()

## Fin.

In [8]:
!git branch

  1-dataframe
  3-group-senses
* 4-semantic-provenance
  dev
  oed-experiments


In [9]:
start,end = 1750,1950

In [10]:
get_last_id = lambda nested_list :[l[-1] for l in nested_list]

In [11]:
query_df = pd.read_pickle(f"./data/senses_{lemma_id}.pickle")

In [12]:
query_sense_ids = query_df.id.unique()

In [13]:
# get all semantic class ids for the sense in seed dataframe
#query_semanticclass_ids = set([sc[-1] for scs in query_df.semantic_class_ids.to_list() for sc in scs])
seeds = [(s,query_oed(credentials,'sense',s,
                        flags=f"current_in='{start}-{end}'&limit=1000"))
                            for s in tqdm(query_sense_ids)]


In [14]:
seeds_df = pd.DataFrame([seed['data'] for s_id,seed in seeds])
seeds_df['provenance'] = seeds_df.id
seeds_df['provenance_type'] = 'seed'
seeds_df.head()

,id,meta,lemma,notes,oed_url,word_id,daterange,first_use,categories,definition,transitivity,oed_reference,quotation_ids,part_of_speech,main_current_sense,semantic_class_ids,provenance,provenance_type
0,machine_nn01-38473945,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38473945,machine_nn01,"{'end': None, 'start': 1545, 'obsolete': False...",J. Schäfer,"{'topic': [], 'region': [], 'register': [['rar...","A material or immaterial structure, esp. the f...",None,"machine, n., sense I.1a","[machine_nn01-38473950, machine_nn01-38473961,...",NN,False,"[[1, 111290, 118635, 119024, 120162, 120172], ...",machine_nn01-38473945,seed
1,machine_nn01-38474233,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38474233,machine_nn01,"{'end': None, 'start': 1583, 'obsolete': False...",Brian Melbancke,"{'topic': [['Military', 'Weaponry']], 'region'...",A military engine or siege-tower. Cf. war mach...,None,"machine, n., sense II.3","[machine_nn01-38474243, machine_nn01-38474252,...",NN,False,"[[153072, 160439, 163207, 163208, 163377, 1633...",machine_nn01-38474233,seed
2,machine_nn01-38474097,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38474097,machine_nn01,"{'end': 1707, 'start': 1595, 'obsolete': True,...",Elizabeth I,"{'topic': [], 'region': [], 'register': []}",spec. A scheme or plot. Obsolete.,None,"machine, n., sense I.1b","[machine_nn01-38474102, machine_nn01-38474122,...",NN,False,"[[1, 84689, 87987, 87988, 87989, 88083, 88109,...",machine_nn01-38474097,seed
3,machine_nn01-38474140,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38474140,machine_nn01,"{'end': None, 'start': 1604, 'obsolete': False...",William Shakespeare,"{'topic': [['Sciences', 'Medicine', 'Anatomy']...","A living body, esp. the human body considered ...",None,"machine, n., sense I.2","[machine_nn01-38474151, machine_nn01-38474160,...",NN,False,"[[1, 8835, 25507, 29189]]",machine_nn01-38474140,seed
4,machine_nn01-38474301,"{'created': 1904, 'revised': True, 'updated': ...",machine,[In early use predominantly a ‘flying’ car alo...,https://www.oed.com/view/Entry/111850#eid38474301,machine_nn01,"{'end': None, 'start': 1609, 'obsolete': False...",Ben Jonson,"{'topic': [['Arts', 'Performing Arts', 'Theatr...",Theatre. A (usually movable) contrivance for t...,None,"machine, n., sense II.4a","[machine_nn01-38474320, machine_nn01-38474329,...",NN,False,"[[153072, 222360, 224508, 232933, 232967, 2330...",machine_nn01-38474301,seed


In [15]:
# get all synonyms for each sense
synonyms = [(s,query_oed(credentials,'sense',s,
                    level='synonyms',
                    flags=f"current_in='{start}-{end}'&limit=1000"))
                            for s in tqdm(query_sense_ids)]

# transform list of synonyms to a dataframe
synonyms_df = pd.DataFrame([s for s_id,syn in synonyms for s in syn['data']])


In [16]:
synonyms_df['provenance'] = [s_id for s_id,syn in synonyms for s in syn['data']]
synonyms_df['provenance_type'] = 'synonym'

In [17]:
# seed + synonyms constitute the nucleas of our query
# branch from there
core_df = pd.concat([seeds_df,synonyms_df])


In [18]:
core_df['semantic_class_last_id'] = core_df['semantic_class_ids'].apply(get_last_id)
core_df.columns

Index(['id', 'meta', 'lemma', 'notes', 'oed_url', 'word_id', 'daterange',
       'first_use', 'categories', 'definition', 'transitivity',
       'oed_reference', 'quotation_ids', 'part_of_speech',
       'main_current_sense', 'semantic_class_ids', 'provenance',
       'provenance_type', 'semantic_class_last_id'],
      dtype='object')

In [19]:
core_df.shape

(511, 19)

In [20]:
quotations_ids = set([q for l in core_df.quotation_ids.to_list() for q in l])
len(quotations_ids)

2710

In [28]:
semantic_class_ids = set([s for l in core_df.semantic_class_last_id.to_list() for s in l])
len(semantic_class_ids)

187

In [25]:
# get semantic class ids of all synonym senses
# the last item of the list in the semantic_class_ids columnns
#synonyms_semanticclass_ids = set([sc[-1] for scs in synonyms_df.semantic_class_ids.to_list() for sc in scs])

In [26]:
# merge semantic class ids of the query senses with synonym senses
#semanticclass_ids = synonyms_semanticclass_ids.union(query_semanticclass_ids)

NameError: name 'query_semanticclass_ids' is not defined

In [30]:
len(semantic_class_ids)

187

In [31]:
# get all the descendants of senses
branches = [(idx,query_oed(credentials,'semanticclass', idx, 
                        level='branchsenses',
                        flags=f"current_in='{start}-{end}'&limit=1000"))
                    for idx in tqdm(semantic_class_ids)]

In [32]:
branches_df = pd.DataFrame([s for idx,branch in branches for s in branch['data']])
branches_df['provenance'] = [idx for idx,branch in branches for s in branch['data']]

In [33]:
branches_df['provenance_type'] = 'branch'
branches_df['semantic_class_last_id'] = branches_df.semantic_class_ids.apply(get_last_id)

In [34]:
branches_df_red = branches_df.loc[~branches_df.id.isin(core_df.id)]
branches_df_red.shape

(6997, 19)

In [35]:
branches_df_red.head()

,id,meta,lemma,notes,oed_url,word_id,daterange,first_use,categories,definition,transitivity,oed_reference,quotation_ids,part_of_speech,main_current_sense,semantic_class_ids,provenance,provenance_type,semantic_class_last_id
0,beachwagon_nn01-25800061,"{'created': 1887, 'revised': False, 'updated':...",beach-wagon,[],https://www.oed.com/view/Entry/16443#eid25800061,beachwagon_nn01,"{'end': None, 'start': 1869, 'obsolete': False...",Louisa May Alcott,"{'topic': [['Transport', 'Motoring']], 'region...","a light open wagon, with two or more seats; a ...",None,"‘beach-wagon’ in beach, n.","[beachwagon_nn01-25800544, beachwagon_nn01-258...",NN,True,"[[153072, 198379, 200757, 201351, 201370, 2016...",202113,branch,"[201845, 202113]"
1,carryall_nn01-9996366,"{'created': 1888, 'revised': False, 'updated':...",carry-all,[],https://www.oed.com/view/Entry/28253#eid9996366,carryall_nn01,"{'end': None, 'start': 1714, 'obsolete': False...",J. Stoddard,"{'topic': [['Crafts and Trades', 'Carriage-bui...","A light carriage for one horse, usually four-w...",None,"carry-all, n., sense 1","[carryall_nn01-9996369, carryall_nn01-30797203...",NN,True,"[[153072, 198379, 200757, 201351, 201370, 2016...",202113,branch,"[201820, 202113]"
2,estatecar_nn01-5268167,"{'created': 1972, 'revised': False, 'updated':...",estate car,[],https://www.oed.com/view/Entry/64556#eid5268167,estatecar_nn01,"{'end': None, 'start': 1950, 'obsolete': False...",Motor Industry,"{'topic': [['Transport', 'Motoring']], 'region...",a light saloon motor car spec. constructed or ...,None,"‘estate car’ in estate, n.","[estatecar_nn01-5268245, estatecar_nn01-526825...",NN,True,"[[153072, 198379, 200757, 201351, 201370, 2018...",202113,branch,[202113]
3,estatewagon_nn01-5268185,"{'created': 1972, 'revised': False, 'updated':...",estate wagon,[],https://www.oed.com/view/Entry/64556#eid5268185,estatewagon_nn01,"{'end': None, 'start': 1959, 'obsolete': False...",Marguerite Steen,"{'topic': [['Transport', 'Motoring']], 'region...",= estate car n.,None,"‘estate wagon’ in estate, n.",[estatewagon_nn01-5268356],NN,True,"[[153072, 198379, 200757, 201351, 201370, 2018...",202113,branch,[202113]
4,ranchwagon_nn01-27047344,"{'created': 1903, 'revised': True, 'updated': ...",ranch wagon,[In quot. 1952 as the name for a specific mode...,https://www.oed.com/view/Entry/157944#eid27047344,ranchwagon_nn01,"{'end': None, 'start': 1879, 'obsolete': False...",Daily Gaz. (Colorado Springs),"{'topic': [['Transport', 'Motoring']], 'region...",(a) a horse-drawn wagon used on a ranch; (b) a...,None,"‘ranch wagon’ in ranch, n.2","[ranchwagon_nn01-121105010, ranchwagon_nn01-27...",NN,True,"[[153072, 198379, 200757, 201351, 201370, 2018...",202113,branch,[202113]


In [36]:
extended_df = pd.concat([core_df,branches_df_red])

In [37]:
extended_df.head()

,id,meta,lemma,notes,oed_url,word_id,daterange,first_use,categories,definition,transitivity,oed_reference,quotation_ids,part_of_speech,main_current_sense,semantic_class_ids,provenance,provenance_type,semantic_class_last_id
0,machine_nn01-38473945,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38473945,machine_nn01,"{'end': None, 'start': 1545, 'obsolete': False...",J. Schäfer,"{'topic': [], 'region': [], 'register': [['rar...","A material or immaterial structure, esp. the f...",None,"machine, n., sense I.1a","[machine_nn01-38473950, machine_nn01-38473961,...",NN,False,"[[1, 111290, 118635, 119024, 120162, 120172], ...",machine_nn01-38473945,seed,"[120172, 120173]"
1,machine_nn01-38474233,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38474233,machine_nn01,"{'end': None, 'start': 1583, 'obsolete': False...",Brian Melbancke,"{'topic': [['Military', 'Weaponry']], 'region'...",A military engine or siege-tower. Cf. war mach...,None,"machine, n., sense II.3","[machine_nn01-38474243, machine_nn01-38474252,...",NN,False,"[[153072, 160439, 163207, 163208, 163377, 1633...",machine_nn01-38474233,seed,[163378]
2,machine_nn01-38474097,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38474097,machine_nn01,"{'end': 1707, 'start': 1595, 'obsolete': True,...",Elizabeth I,"{'topic': [], 'region': [], 'register': []}",spec. A scheme or plot. Obsolete.,None,"machine, n., sense I.1b","[machine_nn01-38474102, machine_nn01-38474122,...",NN,False,"[[1, 84689, 87987, 87988, 87989, 88083, 88109,...",machine_nn01-38474097,seed,[88126]
3,machine_nn01-38474140,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38474140,machine_nn01,"{'end': None, 'start': 1604, 'obsolete': False...",William Shakespeare,"{'topic': [['Sciences', 'Medicine', 'Anatomy']...","A living body, esp. the human body considered ...",None,"machine, n., sense I.2","[machine_nn01-38474151, machine_nn01-38474160,...",NN,False,"[[1, 8835, 25507, 29189]]",machine_nn01-38474140,seed,[29189]
4,machine_nn01-38474301,"{'created': 1904, 'revised': True, 'updated': ...",machine,[In early use predominantly a ‘flying’ car alo...,https://www.oed.com/view/Entry/111850#eid38474301,machine_nn01,"{'end': None, 'start': 1609, 'obsolete': False...",Ben Jonson,"{'topic': [['Arts', 'Performing Arts', 'Theatr...",Theatre. A (usually movable) contrivance for t...,None,"machine, n., sense II.4a","[machine_nn01-38474320, machine_nn01-38474329,...",NN,False,"[[153072, 222360, 224508, 232933, 232967, 2330...",machine_nn01-38474301,seed,[233139]


In [38]:
extended_df.to_pickle('./data/extended_df.pickle')

In [39]:
extended_df = pd.read_pickle('./data/extended_df.pickle')

In [40]:
len(set(extended_df.id))

6925

In [41]:
# sense_idx2quotations = {
#                 sense_idx : query_oed(credentials,'sense',sense_idx,level='quotations')
#                         for sense_idx in tqdm(set(extended_df.id))
#                                 }

# pickle.dump(sense_idx2quotations,open('./data/quotations.pickle','wb'))

In [42]:
sense_idx2quotations = pickle.load(open('./data/quotations.pickle','rb'))

In [43]:
list(sense_idx2quotations.values())[0]['data']

[{'id': 'behaviour_nn01-24139069',
  'text': {'keyword': 'behaviour',
   'full_text': 'All local habitude or behaviour must be between two things or more, in a place so or so.',
   'keyword_offset': 22},
  'year': 1674,
  'lemma': 'behaviour',
  'source': {'title': 'Treat. Bulk & Selvedge',
   'author': 'N. Fairfax',
   'gender': 'male'},
  'oed_url': 'https://www.oed.com/view/Entry/17197#eid24139069',
  'word_id': 'behaviour_nn01',
  'sense_id': 'behaviour_nn01-24139065',
  'datestring': '1674',
  'first_in_word': False,
  'oed_reference': 'behaviour, n., sense 5',
  'first_in_sense': True},
 {'id': 'behaviour_nn01-24139077',
  'text': {'keyword': 'behaviour',
   'full_text': 'In Chemistry the behaviour of different substances towards each other, in respect to combination and affinity.',
   'keyword_offset': 17},
  'year': 1866,
  'lemma': 'behaviour',
  'source': {'title': 'Reign of Law',
   'author': 'Duke of Argyll',
   'gender': 'male'},
  'oed_url': 'https://www.oed.com/view/Entr

In [44]:
df_quot = pd.concat([pd.DataFrame(q['data']) for q in sense_idx2quotations.values()])
df_quot.head()

,id,text,year,lemma,source,oed_url,word_id,sense_id,datestring,first_in_word,oed_reference,first_in_sense
0,behaviour_nn01-24139069,"{'keyword': 'behaviour', 'full_text': 'All loc...",1674,behaviour,"{'title': 'Treat. Bulk & Selvedge', 'author': ...",https://www.oed.com/view/Entry/17197#eid24139069,behaviour_nn01,behaviour_nn01-24139065,1674,False,"behaviour, n., sense 5",True
1,behaviour_nn01-24139077,"{'keyword': 'behaviour', 'full_text': 'In Chem...",1866,behaviour,"{'title': 'Reign of Law', 'author': 'Duke of A...",https://www.oed.com/view/Entry/17197#eid24139077,behaviour_nn01,behaviour_nn01-24139065,1866,False,"behaviour, n., sense 5",False
2,behaviour_nn01-24139085,"{'keyword': 'behaviour', 'full_text': 'To watc...",1878,behaviour,"{'title': 'Physiogr.', 'author': 'T. H. Huxley...",https://www.oed.com/view/Entry/17197#eid24139085,behaviour_nn01,behaviour_nn01-24139065,1878,False,"behaviour, n., sense 5",False
3,behaviour_nn01-24139093,"{'keyword': 'behaviour', 'full_text': 'The beh...",1882,behaviour,"{'title': 'Daily Tel.', 'author': None, 'gende...",https://www.oed.com/view/Entry/17197#eid24139093,behaviour_nn01,behaviour_nn01-24139065,1882,False,"behaviour, n., sense 5",False
0,proceeding_nn01-28488935,"{'keyword': 'procedyng', 'full_text': 'Iche of...",1425,proceeding,"{'title': 'Rolls of Parl.', 'author': None, 'g...",https://www.oed.com/view/Entry/151779#eid28488935,proceeding_nn01,proceeding_nn01-28488926,1425,False,"proceeding, n., sense 2a",True


In [45]:
df_quot.shape

(35246, 12)

In [46]:
merged = extended_df.merge(df_quot[['id','sense_id',"text","year","source"]],left_on='id',right_on='sense_id',suffixes=['',"_quotation"])
merged.head()

,id,meta,lemma,notes,oed_url,word_id,daterange,first_use,categories,definition,...,main_current_sense,semantic_class_ids,provenance,provenance_type,semantic_class_last_id,id_quotation,sense_id,text,year,source
0,machine_nn01-38473945,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38473945,machine_nn01,"{'end': None, 'start': 1545, 'obsolete': False...",J. Schäfer,"{'topic': [], 'region': [], 'register': [['rar...","A material or immaterial structure, esp. the f...",...,False,"[[1, 111290, 118635, 119024, 120162, 120172], ...",machine_nn01-38473945,seed,"[120172, 120173]",machine_nn01-38473950,machine_nn01-38473945,"{'keyword': 'machyne', 'full_text': 'The hole ...",1545,"{'title': 'Early Mod. Eng. Lexicogr.', 'author..."
1,machine_nn01-38473945,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38473945,machine_nn01,"{'end': None, 'start': 1545, 'obsolete': False...",J. Schäfer,"{'topic': [], 'region': [], 'register': [['rar...","A material or immaterial structure, esp. the f...",...,False,"[[1, 111290, 118635, 119024, 120162, 120172], ...",machine_nn01-38473945,seed,"[120172, 120173]",machine_nn01-38473961,machine_nn01-38473945,"{'keyword': 'Machine', 'full_text': 'Machine, ...",1545,"{'title': 'Early Mod. Eng. Lexicogr.', 'author..."
2,machine_nn01-38473945,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38473945,machine_nn01,"{'end': None, 'start': 1545, 'obsolete': False...",J. Schäfer,"{'topic': [], 'region': [], 'register': [['rar...","A material or immaterial structure, esp. the f...",...,False,"[[1, 111290, 118635, 119024, 120162, 120172], ...",machine_nn01-38473945,seed,"[120172, 120173]",machine_nn01-38473973,machine_nn01-38473945,"{'keyword': 'machine', 'full_text': 'The maist...",1550,"{'title': 'Complaynt Scotl.', 'author': None, ..."
3,machine_nn01-38473945,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38473945,machine_nn01,"{'end': None, 'start': 1545, 'obsolete': False...",J. Schäfer,"{'topic': [], 'region': [], 'register': [['rar...","A material or immaterial structure, esp. the f...",...,False,"[[1, 111290, 118635, 119024, 120162, 120172], ...",machine_nn01-38473945,seed,"[120172, 120173]",machine_nn01-38473981,machine_nn01-38473945,"{'keyword': 'machin', 'full_text': 'Be his wis...",1599,"{'title': 'Hymnes', 'author': 'A. Hume', 'gend..."
4,machine_nn01-38473945,"{'created': 1904, 'revised': True, 'updated': ...",machine,[],https://www.oed.com/view/Entry/111850#eid38473945,machine_nn01,"{'end': None, 'start': 1545, 'obsolete': False...",J. Schäfer,"{'topic': [], 'region': [], 'register': [['rar...","A material or immaterial structure, esp. the f...",...,False,"[[1, 111290, 118635, 119024, 120162, 120172], ...",machine_nn01-38473945,seed,"[120172, 120173]",machine_nn01-38473999,machine_nn01-38473945,"{'keyword': 'Machine', 'full_text': 'They that...",1673,"{'title': 'Hist. Quinq-articularis', 'author':..."


In [47]:
exclude_senses = {"machine_nn01-38474097", "machine_nn01-38474140", "machine_nn01-38474301",
                  "machine_nn01-38474405", "machine_nn01-38475994", "machine_nn01-38476096",
                  "machine_nn01-38476316", "machine_nn01-38474974", "machine_nn01-38476397",
                 "machine_nn01-38475013", "machine_nn01-38475772", "machine_nn01-38476566",
                   "machine_nn01-38476245", "machine_nn01-38475835","machine_nn01-38473945"}

include_senses = {'machine_nn01-38474233','machine_nn01-38474548','machine_nn01-38475164','machine_nn01-38475286',
          "machine_nn01-38474607","machine_nn01-38475923","machine_nn01-38474877","machine_nn01-38475046",
          "machine_nn01-38475099"
         }

In [48]:
def select_senses(df,list_senses):
    sc_ids = set([s for l in df.loc[df.id.isin(list_senses)].semantic_class_last_id.to_list() for s in l])
    print(sc_ids)
    print(df.shape)
    overlap = lambda x,l: bool(set(x).intersection(l))
    
    df = df.loc[(df.id.isin(list_senses))  | (df.provenance.isin(list_senses)) | (df.provenance.isin(sc_ids))]
    print(df.shape)
    return df

df = select_senses(merged,include_senses)

{'201575', '214198', '214572', '202100', '208136', '212078', '202575', '206654', '88147', '202421', '163378', '202530', '75661'}
(38749, 24)
(4109, 24)


In [49]:
len(df.lemma.unique())

759

In [50]:
df.lemma.value_counts()

machine           180
fleet              34
boat               34
set                32
vessel             28
                 ... 
nacelle             1
mail diligence      1
post-equipage       1
boatage             1
battle-ram          1
Name: lemma, Length: 759, dtype: int64

In [51]:
# def filter_by_senses(df,list_senses):
#     sc_ids = set([s for l in df.loc[df.id.isin(list_senses)].semantic_class_last_id.to_list() for s in l])
#     print(sc_ids)
#     overlap = lambda x,l: bool(set(x).intersection(l))
#     # remove those for which id appears in list_senses
#     df = df.loc[~df.id.isin(list_senses)]
#     print(df.shape)
#     # remove synonyms for which the provenance list an id for list_senses
#     df = df.loc[~df.provenance.isin(list_senses)]
#     df = df.loc[~df.provenance.isin(sc_ids)]

#     print(df.shape)
#     return df

# df = filter_by_senses(extended_df,exclude_senses)

In [52]:
df_sub = df[df.provenance_type.isin(['seed','synonym'])]
df_sub.shape

(1113, 24)

In [53]:
import numpy as np

In [91]:
def process_quotations(text_col):
    
    punct = [',','.',' ','?','!']
    sentence = text_col["full_text"]
    
    
    
    sentence = np.array([i for i in sentence])
    offset = text_col["keyword_offset"]
    target = text_col["keyword"]
    
    if not target:
        return None
    
    labels = np.array([0]*len(sentence))
    end = offset + len(target)
    labels[offset:end] = 1
    
    for ch in punct:
        labels[np.where(sentence==ch)] = 2
    
    rows = []
    word,labs = [],[]
    
    for i in range(len(sentence)):
        if labels[i] < 2:
            word.append(sentence[i])
            labs.append(labels[i])
        
        if labels[i] == 2 and word:
            rows.append((''.join(word),{0:"notmachine",1:"machine"}[list(set(labs))[0]]))
            word,labs = [],[]
    return rows
    


In [92]:
df_sub["tagged"] = df_sub.text.apply(process_quotations)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
df_sub['partition'] = [np.random.choice(['train','dev','test'], p=[0.6, 0.2, 0.2]) for _ in range(df_sub.shape[0])]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
df_train = df_sub.loc[df_sub.partition=="train"]

df_test = df_sub.loc[df_sub.partition=="test"]

df_dev = df_sub.loc[df_sub.partition=="dev"]

In [99]:
def df2string(df):
    return "\n\n".join(['\n'.join(['\t'.join(e) for e in l]) for l in df.tagged.to_list() if l])

train,test,dev = df2string(df_train),df2string(df_test),df2string(df_dev)

In [100]:
with open('./data/train.csv','w') as out_doc:
    out_doc.write(train)
with open('./data/test.csv','w') as out_doc:
    out_doc.write(test)
with open('./data/dev.csv','w') as out_doc:
    out_doc.write(dev)

In [101]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {0: 'text', 1: 'label'}


path = Path("./data")
corpus: Corpus = ColumnCorpus(path, columns,
                              train_file=f'train.csv',
                              test_file=f'test.csv',
                              dev_file= f'dev.csv',skip_first_line=False)

2020-10-01 22:50:41,512 Reading data from data
2020-10-01 22:50:41,513 Train: data/train.csv
2020-10-01 22:50:41,514 Dev: data/dev.csv
2020-10-01 22:50:41,515 Test: data/test.csv


In [102]:
# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='label')
print(tag_dictionary)


Dictionary with 6 tags: <unk>, O, notmachine, machine, <START>, <STOP>


In [103]:
from flair.data import Corpus
from flair.datasets import UD_ENGLISH
from flair.embeddings import WordEmbeddings,FlairEmbeddings,StackedEmbeddings,TransformerWordEmbeddings




# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

#embeddings = TransformerWordEmbeddings('bert-base-cased',fine_tune=True, allow_long_sentences=True)
    
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='label',
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-pos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

I1001 22:51:01.743118 4495916480 utils.py:422] loading Word2VecKeyedVectors object from /Users/kbeelen/.flair/embeddings/glove.gensim
I1001 22:51:03.038181 4495916480 utils.py:461] loading vectors from /Users/kbeelen/.flair/embeddings/glove.gensim.vectors.npy with mmap=None
I1001 22:51:03.200335 4495916480 utils.py:494] setting ignored attribute vectors_norm to None
I1001 22:51:03.201395 4495916480 utils.py:428] loaded /Users/kbeelen/.flair/embeddings/glove.gensim


2020-10-01 22:51:03,756 ----------------------------------------------------------------------------------------------------
2020-10-01 22:51:03,757 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 256, batch_first=True, b

2020-10-01 22:53:57,573 epoch 5 - iter 2/21 - loss 2.46185231 - samples/sec: 38.59 - lr: 0.100000
2020-10-01 22:53:59,230 epoch 5 - iter 4/21 - loss 2.47754264 - samples/sec: 38.65 - lr: 0.100000
2020-10-01 22:54:00,470 epoch 5 - iter 6/21 - loss 2.43438727 - samples/sec: 51.64 - lr: 0.100000
2020-10-01 22:54:01,706 epoch 5 - iter 8/21 - loss 2.51417036 - samples/sec: 51.83 - lr: 0.100000
2020-10-01 22:54:03,043 epoch 5 - iter 10/21 - loss 2.49188563 - samples/sec: 47.91 - lr: 0.100000
2020-10-01 22:54:04,695 epoch 5 - iter 12/21 - loss 2.52899115 - samples/sec: 38.77 - lr: 0.100000
2020-10-01 22:54:05,878 epoch 5 - iter 14/21 - loss 2.55748762 - samples/sec: 54.15 - lr: 0.100000
2020-10-01 22:54:07,167 epoch 5 - iter 16/21 - loss 2.55844019 - samples/sec: 49.71 - lr: 0.100000
2020-10-01 22:54:08,565 epoch 5 - iter 18/21 - loss 2.56209676 - samples/sec: 45.81 - lr: 0.100000
2020-10-01 22:54:09,725 epoch 5 - iter 20/21 - loss 2.50262889 - samples/sec: 55.20 - lr: 0.100000
2020-10-01 22:

2020-10-01 22:55:42,502 ----------------------------------------------------------------------------------------------------
2020-10-01 22:55:42,503 EPOCH 10 done: loss 2.0668 - lr 0.1000000
2020-10-01 22:55:43,935 DEV : loss 2.112274169921875 - score 0.945
2020-10-01 22:55:43,949 BAD EPOCHS (no improvement): 1
2020-10-01 22:55:43,951 ----------------------------------------------------------------------------------------------------
2020-10-01 22:55:45,726 epoch 11 - iter 2/21 - loss 2.26883578 - samples/sec: 36.08 - lr: 0.100000
2020-10-01 22:55:46,952 epoch 11 - iter 4/21 - loss 2.17234075 - samples/sec: 52.25 - lr: 0.100000
2020-10-01 22:55:48,284 epoch 11 - iter 6/21 - loss 2.06990910 - samples/sec: 48.12 - lr: 0.100000
2020-10-01 22:55:49,657 epoch 11 - iter 8/21 - loss 2.01598373 - samples/sec: 46.64 - lr: 0.100000
2020-10-01 22:55:50,921 epoch 11 - iter 10/21 - loss 2.00928240 - samples/sec: 50.66 - lr: 0.100000
2020-10-01 22:55:52,187 epoch 11 - iter 12/21 - loss 1.96444810 - 

2020-10-01 22:57:27,150 epoch 16 - iter 14/21 - loss 1.52225122 - samples/sec: 60.96 - lr: 0.050000
2020-10-01 22:57:28,512 epoch 16 - iter 16/21 - loss 1.54157100 - samples/sec: 47.01 - lr: 0.050000
2020-10-01 22:57:29,790 epoch 16 - iter 18/21 - loss 1.53144450 - samples/sec: 50.14 - lr: 0.050000
2020-10-01 22:57:31,143 epoch 16 - iter 20/21 - loss 1.52337828 - samples/sec: 47.33 - lr: 0.050000
2020-10-01 22:57:32,043 ----------------------------------------------------------------------------------------------------
2020-10-01 22:57:32,044 EPOCH 16 done: loss 1.5351 - lr 0.0500000
2020-10-01 22:57:33,378 DEV : loss 1.7759202718734741 - score 0.9521
2020-10-01 22:57:33,391 BAD EPOCHS (no improvement): 2
2020-10-01 22:57:33,394 ----------------------------------------------------------------------------------------------------
2020-10-01 22:57:34,887 epoch 17 - iter 2/21 - loss 1.57025582 - samples/sec: 42.95 - lr: 0.050000
2020-10-01 22:57:36,153 epoch 17 - iter 4/21 - loss 1.5618329

{'test_score': 0.9569,
 'dev_score_history': [0.9369,
  0.9389,
  0.9355,
  0.9386,
  0.9456,
  0.9463,
  0.947,
  0.9504,
  0.9507,
  0.945,
  0.9463,
  0.944,
  0.9443,
  0.9541,
  0.9534,
  0.9521],
 'train_loss_history': [6.751686970392863,
  3.0335018634796143,
  2.7342559610094344,
  2.4917304004941667,
  2.4838735659917197,
  2.214115239325024,
  2.1624793154852733,
  2.19656803494408,
  2.062829772631327,
  2.066799657685416,
  1.9141425632295155,
  1.8648326510474795,
  1.8013603857585363,
  1.5993040459496635,
  1.6094313263893127,
  1.5350978601546514],
 'dev_loss_history': [2.6299357414245605,
  2.370863199234009,
  2.890554189682007,
  2.7716479301452637,
  2.063163995742798,
  2.0206449031829834,
  2.0271503925323486,
  1.9765294790267944,
  1.9289461374282837,
  2.112274169921875,
  1.9838744401931763,
  2.049699306488037,
  2.1111295223236084,
  1.7752426862716675,
  1.7800581455230713,
  1.7759202718734741]}